### Pig: Índice invertido

Partiendo del dataset de posts utilizado anteriormente, vamos a calcular un índice invertido. 

In [1]:
! mkdir -p pig-indiceinvertido
import os
os.chdir("pig-indiceinvertido/")
! pwd

/media/sf_jmalvarezf/nextcloud/Documentos/Master/Bloque_3/Pig/ejercicio/pig-indiceinvertido


In [60]:
! dos2unix ../dataset/forum_node.tsv

dos2unix: converting file ../dataset/forum_node.tsv to UNIX format ...


In [64]:
! hadoop fs -put ../dataset/forum_node.tsv

In [4]:
! hadoop fs -put ../dataset/forum1.tsv

In [81]:
%%writefile test.pig
REGISTER /usr/lib/pig/piggybank.jar;
DEFINE StringToInt InvokeForInt('java.lang.Integer.valueOf', 'String');
data = load 'forum_node.tsv' using org.apache.pig.piggybank.storage.CSVExcelStorage('\t', 'YES_MULTILINE', 'NOCHANGE', 'SKIP_INPUT_HEADER') as (pid:chararray, title:chararray, tagnames:chararray, author_id:chararray,
                                                    body:chararray, node_type:chararray, parent_id:chararray, abs_parent_id:chararray,
                                                    added_at:chararray, score:chararray, state_string:chararray, 
                                                    last_edited_id:chararray, last_activity_by_id:chararray, 
                                                    last_activity_at:chararray, active_revision_id:chararray, extra:chararray,
                                                    extra_ref_id:chararray, extra_count:chararray, marked:chararray);
cleandata = foreach data generate REPLACE(pid, '[a-zA-Z]+', '') as post_id, LOWER(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(body, '\\\\n\\\\r', ''), '\\\\r', ''), '\\\\n', ''), '<*>', ''), '[^a-zA-Z0-9\'\\s]+', ' ')) AS clean_body;
cleandata_filtered = filter cleandata by org.apache.pig.piggybank.evaluation.IsNumeric(post_id);
words_data = FOREACH cleandata_filtered GENERATE StringToInt(post_id) as post_id_int:int, FLATTEN(TOKENIZE(clean_body)) as word;
words_data_filtered = filter words_data by SIZE(word) > 0;
word_groups = GROUP words_data_filtered BY word;
index = FOREACH word_groups {
pairs = DISTINCT $1.$0;
cnt = COUNT(pairs);
GENERATE $0 as word, pairs as index_bag, cnt as count;
};
sorted_index = foreach index {
    sorted_bag = order index_bag by $0;
    generate word, sorted_bag, count;
}
STORE sorted_index INTO 'inverted_index';

/* 
Pasos:
1.Carga el fichero de los posts forum_node.tsv, utilizando una extension de Piggybank para poder quitar la cabecera,
en vez de usar directamente el PigStorage.
2.Limpiamos el fichero quitando los saltos de linea, expresiones html y la expresión regular que se proponia en el ejercicio.
También quitamos los caracteres que no son numeros en el post_id.
3.Filtramos los datos de post_id que no son numericos.
4.Creamos tuplas separando el body por espacios y convirtiendo el post_id en un numerico a través de una función custom, 
para evitar problemas que sufrimos con el cast de String a Integer.
5.Agrupamos por palabra
6.Por cada grupo de palabras, hacemos un distinct para los post_id, eliminando los duplicados, contamos el número de post
en que aparece (despues de quitar los duplicados) y generamos una fila con el índice.
7.Como se pide que el indice lleve el post_id ordenador, ordenamos la bag resultante de los posts por su id.
8. Lo guardamos en un fichero.
*/





Overwriting test.pig


In [83]:
! pig -f test.pig -x local

log4j:WARN No appenders could be found for logger (org.apache.hadoop.util.Shell).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
2018-01-28 14:42:27,648 [main] INFO  org.apache.pig.Main - Apache Pig version 0.12.0-cdh5.12.0 (rexported) compiled Jun 29 2017, 04:34:31
2018-01-28 14:42:27,648 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/sf_jmalvarezf/nextcloud/Documentos/Master/Bloque_3/Pig/ejercicio/pig-indiceinvertido/pig_1517146947627.log
2018-01-28 14:42:27,666 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2018-01-28 14:42:28,461 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /home/cloudera/.pigbootup not found
2018-01-28 14:42:28,659 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-28 14:42:28

2018-01-28 14:42:31,107 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 1 map-reduce job(s) waiting for submission.
2018-01-28 14:42:31,108 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker.http.address is deprecated. Instead, use mapreduce.jobtracker.http.address
2018-01-28 14:42:31,128 [JobControl] INFO  org.apache.hadoop.metrics.jvm.JvmMetrics - Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
2018-01-28 14:42:31,419 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2018-01-28 14:42:31,473 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input paths to process : 1
2018-01-28 14:42:31,473 [JobControl] INFO  org.apache.pig.backend.hadoop.executionengine.util.MapRedUtil - Total input paths to process : 1
2018-01-28 14:42:31,501 [JobC

2018-01-28 14:42:46,452 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Finished spill 1
2018-01-28 14:42:46,462 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Merger - Merging 2 sorted segments
2018-01-28 14:42:46,479 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Merger - Down to the last merge-pass, with 2 segments left of total size: 60032390 bytes
2018-01-28 14:42:49,377 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local349806129_0001_m_000000_0 is done. And is in the process of committing
2018-01-28 14:42:49,378 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - map > sort
2018-01-28 14:42:49,378 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task - Task 'attempt_local349806129_0001_m_000000_0' done.
2018-01-28 14:42:49,378 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner 

2018-01-28 14:43:13,015 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Spilling map output
2018-01-28 14:43:13,015 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - bufstart = 0; bufend = 40677656; bufvoid = 104857600
2018-01-28 14:43:13,015 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - kvstart = 26214396(104857584); kvend = 15412296(61649184); length = 10802101/6553600
2018-01-28 14:43:13,015 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - (EQUATOR) 51163412 kvi 12790848(51163392)
2018-01-28 14:43:17,098 [SpillThread] INFO  org.apache.hadoop.mapred.MapTask - Finished spill 0
2018-01-28 14:43:17,098 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - (RESET) equator 51163412 kv 12790848(51163392) kvi 10169420(40677680)
2018-01-28 14:43:18,581 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - 
2018-01-

2018-01-28 14:43:33,582 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.LocalFetcher - localfetcher#1 about to shuffle output of map attempt_local349806129_0001_m_000003_0 decomp: 45856899 len: 45856903 to MEMORY
2018-01-28 14:43:33,666 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.InMemoryMapOutput - Read 45856899 bytes from map-output for attempt_local349806129_0001_m_000003_0
2018-01-28 14:43:33,670 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.MergeManagerImpl - closeInMemoryFile -> map-output of size: 45856899, inMemoryMapOutputs.size() -> 1, commitMemory -> 0, usedMemory ->45856899
2018-01-28 14:43:33,706 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.LocalFetcher - localfetcher#1 about to shuffle output of map attempt_local349806129_0001_m_000001_0 decomp: 67100522 len: 67100526 to MEMORY
2018-01-28 14:43:33,828 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.InMemoryMapOutput - Read 67100522 bytes f

2018-01-28 14:45:14,313 [main] WARN  org.apache.pig.tools.pigstats.PigStatsUtil - Failed to get RunningJob for job job_local349806129_0001
2018-01-28 14:45:14,333 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 100% complete
2018-01-28 14:45:14,333 [main] INFO  org.apache.pig.tools.pigstats.SimplePigStats - Detected Local mode. Stats reported below may be incomplete
2018-01-28 14:45:14,345 [main] INFO  org.apache.pig.tools.pigstats.SimplePigStats - Script Statistics: 

HadoopVersion	PigVersion	UserId	StartedAt	FinishedAt	Features
2.6.0-cdh5.12.0	0.12.0-cdh5.12.0	cloudera	2018-01-28 14:42:30	2018-01-28 14:45:14	GROUP_BY,FILTER

Success!

Job Stats (time in seconds):
JobId	Alias	Feature	Outputs
job_local349806129_0001	1-1,cleandata,cleandata_filtered,data,index,pairs,sorted_bag,sorted_index,word_groups,words_data,words_data_filtered	GROUP_BY,DISTINCT	file:///media/sf_jmalvarezf/nextcloud/Documentos/Master/Bloque_3/Pig/ejercicio/pig-indiceinve

In [91]:
! tail -40000 ./inverted_index/part-r-00000  | head -10

	{(2010230)}	1
reciprocals	{(2009620),(2009707),(2010947),(2014292),(9002395)}	5
reciprocate	{(2001902),(6004447)}	2
reciprocity	{(2008341),(2009563),(2009571),(2009804),(2010292)}	5
recitations	{(2018236),(5014249)}	2
reclamation	{(10438)}	1
recliningon	{(6022210)}	1
recognisers	{(5007198),(5007700),(5007756)}	3
recognising	{(1014977),(1034000),(6015555),(8000513)}	4
recognition	{(3982),(5301),(8066),(9102),(22789),(41152),(47260),(51450),(51559),(52330),(53802),(53881),(53962),(60657),(60916),(63420),(64321),(64470),(66804),(66874),(67092),(67121),(67242),(67488),(1000219),(1000901),(1001733),(1001920),(1002371),(1005129),(1006927),(1007272),(1008146),(1008955),(1008996),(1009010),(1009776),(1010107),(1010329),(1010351),(1012085),(1012836),(1013518),(1013848),(1014107),(1015692),(1018390),(1023592),(1025649),(1026946),(1028196),(1030214),(1030460),(1030646),(1030651),(1031191),(1031238),(1031734),(1032697),(1032720),(1033371),(1033635),(1033857),(1034081),(1034451),(1034946),(1034954

In [86]:
! pig -f test.pig

log4j:WARN No appenders could be found for logger (org.apache.hadoop.util.Shell).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
2018-01-28 14:45:42,674 [main] INFO  org.apache.pig.Main - Apache Pig version 0.12.0-cdh5.12.0 (rexported) compiled Jun 29 2017, 04:34:31
2018-01-28 14:45:42,675 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/sf_jmalvarezf/nextcloud/Documentos/Master/Bloque_3/Pig/ejercicio/pig-indiceinvertido/pig_1517147142644.log
2018-01-28 14:45:44,156 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /home/cloudera/.pigbootup not found
2018-01-28 14:45:44,351 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2018-01-28 14:45:44,352 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-

2018-01-28 14:45:54,325 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 1 map-reduce job(s) waiting for submission.
2018-01-28 14:45:54,326 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker.http.address is deprecated. Instead, use mapreduce.jobtracker.http.address
2018-01-28 14:45:54,326 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2018-01-28 14:45:54,334 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2018-01-28 14:45:54,359 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-28 14:45:54,911 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input paths to process : 1
2018-01-28 14:45:54,911 [JobControl] INFO  org.apache.pig.backend.hadoop.execut

In [87]:
! hadoop fs -ls inverted_index/

Found 2 items
-rw-r--r--   1 cloudera cloudera          0 2018-01-28 14:48 inverted_index/_SUCCESS
-rw-r--r--   1 cloudera cloudera   86936662 2018-01-28 14:48 inverted_index/part-r-00000


In [90]:
! hadoop fs -cat inverted_index/part-r-00000 | tail -40000 | head -10

	{(2010230)}	1
reciprocals	{(2009620),(2009707),(2010947),(2014292),(9002395)}	5
reciprocate	{(2001902),(6004447)}	2
reciprocity	{(2008341),(2009563),(2009571),(2009804),(2010292)}	5
recitations	{(2018236),(5014249)}	2
reclamation	{(10438)}	1
recliningon	{(6022210)}	1
recognisers	{(5007198),(5007700),(5007756)}	3
recognising	{(1014977),(1034000),(6015555),(8000513)}	4
recognition	{(3982),(5301),(8066),(9102),(22789),(41152),(47260),(51450),(51559),(52330),(53802),(53881),(53962),(60657),(60916),(63420),(64321),(64470),(66804),(66874),(67092),(67121),(67242),(67488),(1000219),(1000901),(1001733),(1001920),(1002371),(1005129),(1006927),(1007272),(1008146),(1008955),(1008996),(1009010),(1009776),(1010107),(1010329),(1010351),(1012085),(1012836),(1013518),(1013848),(1014107),(1015692),(1018390),(1023592),(1025649),(1026946),(1028196),(1030214),(1030460),(1030646),(1030651),(1031191),(1031238),(1031734),(1032697),(1032720),(1033371),(1033635),(1033857),(1034081),(1034451),(1034946),(1034954